아래와 같은 상황을 가정하자. 

$X_i = \mu_i +\epsilon_i.$

단 여기에서 $\epsilon_i \sim N(0,1)$ 이다. 이러한 문제는, 예를들어, 천문학 이미지 프로세싱, 데이터마이닝, 모형선택, 웨이블릿 혹은 다른 dictionaries에 의한 함수추정 문제에서 발생한다. (따라서 이 논문은 쓸모가 있다) 언급한 예시에 대한 자세한 discussion은 Johnstone and Silverman (2004)를 참고하면 된다. 

많은 실제예제에서 $\mu_i$는 어떤 의미에서 (in some sense) sparse하다고 여길 수 있다. EbayesThresh 패키지는 이처럼 $\mu_i$가 sparse하다는 구조 (혹은 가정)을 이용하여 $\mu_i$를 적절하게 추정한다. 


***Sparsity***를 이용하는 자연스러운 방법은 ***threshoding***이다: 여기에서 threshold의 값 $t$를 너무 크게 잡으면 신호를 잡음으로 잘못 판단할 것이고 $t$의 값이 너무 작다면 잡음을 신호로 잘못 판단할 수 있다. 따라서 $t$의 선택은 이 양쪽 기준사이의 tradeoff가 있는데 EbayesThresh는 이러한 tradeoff를 자동으로 조정하는 효과가 있다.

- $\mu_i$는 $w$의 확률로 0 이며 $(1-w)$의 확률로 0이 아니다. $\mu_i$가 0이 아닐경우에는 symmetric heavy-tailed density $\gamma$에서 추출된다고 가정한다. 여기에서 prior에 대한 key parameter인 $w$는 데이터로부터 자동으로 추정된다. (marginal maximum likelihood 를 이용한다) 그리고 추정된 $w$는 Bayesian model로 다시 대입된다. 

- $w$가 추정되면 Bayesian model은 thresholding procedure를 수행할 수 있다. 왜냐하면 $w$를 추정하면 $t(w)$를 선택한다는 말과 같은말이기 때문이다.

**argument**

`prior`: $\mu_i$의 density. 보통 $\frac{1}{2}a \exp(-a|u|)$라고 가정한다. parameter $a$는 Section 2.1에서 자시해 나옴. 

`bayesfac`, `threshrule`: Section 2.2, 2.3에 자세히 나온다. 

`sdev`: $X_i-\mu_i$의 noise를 의미한다. 이 값은 보통 observed data로 부터 추정하는데 보통 $median(|X_i|)$로 추정한다. 이것의 motivation은 (the movtivation for this is that) 비록 $\mu_i$가 합리적인 수준에서 sparse하더라도 median absolute value는 영향을 받지 않기 때문이다. 당연히 신호가 sparse하지 않다면 이러한 방식으로 `sdev`를 추정하는 것은 매우 조심스럽게 수행되어야 할 것이다. 만약에 `sdev`를 알고있는 경우라면 적절하게 추정하면 된다. 

# Description of the method 

## The Bayesian model 

$X_i \sim N(\mu_i,1)$

$f_{\text{prior}}(\mu)=(1-w)\delta_0(\mu)+w \gamma_a(\mu), \quad \gamma_a(\mu)=\frac{1}{2}a\exp(-a|\mu|)$

여기에서 $\gamma_a(\mu)$는 하나의 예시일 뿐이다. Ebayesthresh에 디폴트로 설정된 `prior="laplace"`를 셋팅하면 $\gamma_a(\mu)$가 사용된다. $\gamma$의 선택은 tail이 polynomial rates로 줄어드는 어떠한 분포를 사용해도 무방하다. 저자들은 quasi-Cauchy분포를 제안하였는데 이는 Johnstone and Sliverman이 만든 theoretical assumption을 만족하는 분포중 가장 꼬리가 두꺼운 분포이다. 

## Thresholding rules 

모수 $\mu$는 사전분포(prior distribution)를 가진다고 가정하고 $X \sim N(\mu,1)$이라고 가정하자. 이 경우 $X=x$가 given되었을 경우 $\mu$의 사후분포(posterior distribution)를 구할 수 있다. (자세한 내용은 Section 6을 참고해야함) 사후분포의 중앙값을 $\hat{\mu}(x;w)$라고 하자. (사후분포의 중앙값이 $w$에 영향받는 이유는 사전분포가 $w$에 depend하기 때문이다. 여기에서 $w$는 marginal MLE로 적절히 추론한다고 가정한다) 

$X_i$는 독립이라고 가정한다. 여기에서 $X_i$가 독립이 아니라면 약간의 정보손실이 있을 수 있다. 하지만 $X_i$ 사이에 너무 많은 dependency가 존재하는 경우가 아니라면 Ebayesthresh는 어느정도 합리적인 결과를 제공한다. 

만약에 `bayesfac=TRUE`를 사용하면 $\mu$의 사후분포의 중앙값 대신에 Bayes factor threshold 를 쓸 수도 있다. 

## Choosing the threshold

$X_i$의 marginal density는 

$(1-w)\phi(x) +w(\gamma \star \phi)(x)$

$l(w) = \sum_{i=1}^{n}\log \big\{(1-w)\phi(X_i)+wg(X_i) \big\}$ 

와 같이 정의가능하다. 단, 여기에서 $g:= \gamma\star \phi$ 이다. 

이제 우리는 아래의 식을 풀면된다. 

$$\underset{w}{\operatorname{argmax}} l(w)\quad\quad \text{subject to}\quad t(w) \leq \sqrt{2\log n}$$

여기에서 $\sqrt{2\log n}$은 흔히 말하는 universal threshold 이다. 

만약에 $w$이외에 $a$도 추정해야 한다면 아래와 같이 추정할 수 있다. 

$$\underset{w}{\operatorname{argmax}} l(w)\quad\quad \text{subject to}\quad t(w) \leq \sqrt{2\log n}$$